In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import xmltodict
#import urllib2
from xml.etree import ElementTree as ET
import http.client 
import api_keys

# 1. Get School Info

In [6]:
requestURL= "https://api.greatschools.org/schools/TX/Austin?key=" + api_keys.SchoolAPI + "&limit=500"

r = requests.get(requestURL)
root = ET.fromstring(r.text)
tree = ET.ElementTree(root)
tree.write("SchoolData.xml")

In [7]:
e = ET.parse('SchoolData.xml').getroot()

In [8]:
#define a school dataframe
schools_df = pd.DataFrame()
schools_df["gsId"] = np.nan
schools_df["name"] = np.nan
schools_df["city"] = np.nan
schools_df["state"] = np.nan
schools_df["address"] = np.nan
schools_df["lat"] = np.nan
schools_df["lon"] = np.nan
schools_df["type"] = np.nan
schools_df["gsRating"] = 0
schools_df["Zipcode"] = np.nan


with open('files.xml',encoding="utf8") as datafile:
        doc = xmltodict.parse(datafile.read())
index = 0

for row in doc['schools']['school']:
    try:
        schools_df.loc[index, "gsId"] = row["gsId"]
        schools_df.loc[index, "name"] = row["name"]
        schools_df.loc[index, "city"] = row["city"]
        schools_df.loc[index, "state"] = row["state"]
        schools_df.loc[index, "address"] = row["address"]
        schools_df.loc[index, "lat"] = row["lat"]
        schools_df.loc[index, "lon"] = row["lon"]
        schools_df.loc[index, "type"] = row["type"]
        schools_df.loc[index, "Zipcode"] = row["address"][-5:]
        index = index+1
        #print(f'gsId: {row["gsId"]}, name: {row["name"]}')
    except Exception:
        pass
    try:
        schools_df.loc[index, "gsRating"] = row["gsRating"]
    except:
        pass
    
schools_df["gsRating"] = schools_df[["gsRating"]].apply(pd.to_numeric)

In [9]:
schools_df = schools_df.dropna(how='all',axis=0) 
schools_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 349
Data columns (total 10 columns):
gsId        349 non-null object
name        349 non-null object
city        349 non-null object
state       349 non-null object
address     349 non-null object
lat         349 non-null object
lon         349 non-null object
type        349 non-null object
gsRating    220 non-null float64
Zipcode     349 non-null object
dtypes: float64(1), object(9)
memory usage: 30.1+ KB


In [10]:
schools_df.to_csv("Schooldata.csv")

In [11]:
schools_df[(schools_df["gsRating"] >7)]

,gsId,name,city,state,address,lat,lon,type,gsRating,Zipcode
13,5869,Anderson Mill Elementary School,Austin,TX,"10610 Salt Mill Hollow, \nAustin, TX 78750",30.452835,-97.806854,public,8.0,78750
34,7781,Bannockburn Christian Academy,Austin,TX,"7100 Brodie Ln, \nAustin, TX 78745",30.212337,-97.8324,private,8.0,78745
36,523,Barrington Elementary School,Austin,TX,"400 Cooper Drive, \nAustin, TX 78753",30.360529,-97.69607,public,9.0,78753
38,524,Barton Hills Elementary School,Austin,TX,"2108 Barton Hills Drive, \nAustin, TX 78704",30.254395,-97.783356,public,10.0,78704
39,2035,Baty Elementary School,Austin,TX,"2101 Faro Drive, \nAustin, TX 78741",30.227068,-97.712814,public,9.0,78741
43,527,Blackshear Elementary School,Austin,TX,"1712 East 11th Street, \nAustin, TX 78702",30.266968,-97.72217,public,8.0,78702
44,528,Blanton Elementary School,Austin,TX,"5408 Westminster Drive, \nAustin, TX 78723",30.30693,-97.69063,public,8.0,78723
49,613,Bowie High School,Austin,TX,"4103 West Slaughter Lane, \nAustin, TX 78749",30.186369,-97.85873,public,8.0,78749
50,12308,Boys Menachem Hebrew Academy,Austin,TX,"7010 Village Center Dr, \nAustin, TX 78731",30.35242,-97.7568,private,9.0,78731
55,530,Brooke Elementary School,Austin,TX,"3100 East 4th Street, \nAustin, TX 78702",30.254822,-97.70851,public,10.0,78702


# 2. Get the property rate

In [191]:
#define a house Data frame

house_df = pd.DataFrame()
house_df['Zipcode'] = np.nan
house_df['avgsaleprice'] = np.nan
house_df['medsaleprice'] = np.nan
house_df['homesalecount'] = np.nan

In [192]:
conn = http.client.HTTPSConnection("search.onboard-apis.com") 

headers = { 
    'accept': "application/json", 
    'apikey': api_keys.HouseAPI, 
} 
searchstring = "/propertyapi/v1.0.0/salestrend/snapshot?"
serchvalues = "&interval=yearly&startyear=2017&endyear=2018"


index = 0

for zipcode in schools_df["Zipcode"]:
    print(zipcode)
    # Starting URL for Weather Map API Call
    geoid = "geoid=ZI" + zipcode
    conn.request("GET", searchstring+geoid+serchvalues, headers=headers) 
    data = json.loads(conn.getresponse().read())
    
    #load DF
    try:
        house_df.loc[index, 'Zipcode'] = zipcode
        house_df.loc[index, 'avgsaleprice'] = data['salestrends'][0]['SalesTrend']['avgsaleprice']
        house_df.loc[index, 'medsaleprice'] = data['salestrends'][0]['SalesTrend']['medsaleprice']
        house_df.loc[index, 'homesalecount'] = data['salestrends'][0]['SalesTrend']['homesalecount'] 
        
        index = index +1
    except:
        pass
    #print(data)
    #break;
    
house_df

78732
78756
78737
78751
78733
78722
78746
78748
78702
78741
78744
78702
78759
78750
78723
78728
78723
78748
78709
78724
78703
78727
78731
78745
78745
78745
78723
78752
78703
78704
78737
78739
78727
78739
78745
78748
78753
78733
78704
78741
78704
78745
78738
78702
78723
78744
78734
78754
78749
78749
78731
78753
78753
78757
78746
78702
78752
78703
78757
78722
78726
78732
78759
78745
78756
78748
78703
78759
78701
78746
78717
78752
78753
78736
78717
78728
78741
78739
78722
78758
78753
78749
78748
78747
78745
78745
78704
78725
78727
78704
78724
78729
78754
78723
78754
78754
78753
78731
78721
78745
78729
78717
78746
78753
78729
78750
78704
78746
78704
78724
78702
78702
78703
78749
78617
78753
78726
78723
78751
78753
78757
78744
78744
78717
78758
78753
78723
78753
78701
78751
78731
78750
78746
78759
78744
78717
78753
78750
78723
78717
78758
78744
78738
78756
78751
78759
78751
78741
78753
78753
78741
78744
78744
78721
78731
78734
78748
78728
78753
78744
78729
78724
78724
78745
78701
78724
7874

TypeError: must be str, not float

In [194]:
house_df.to_csv("Housingdata.csv")
